# 🔍 Scraper Wikidata - Recherche d'entités

**Auteur:** May8326  
**Date:** 2025-07-02  
**Description:** Outil pour scraper toutes les entités Wikidata contenant un terme de recherche spécifique.

Ce notebook utilise l'API EntitySearch de Wikidata avec pagination automatique pour récupérer **tous** les résultats sans limitation des 50 premiers.

---

## 📦 Installation et imports

Installation des dépendances nécessaires :

In [ ]:
# Installation des dépendances (décommentez si nécessaire)
# !pip install requests pandas jupyter

In [14]:
import requests
import time
import json
import csv
import pandas as pd
from urllib.parse import quote
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

print("✅ Toutes les bibliothèques sont importées avec succès!")

✅ Toutes les bibliothèques sont importées avec succès!


## ⚙️ Configuration

Définissez vos paramètres de recherche ici :

In [15]:
# Configuration globale
LANGUAGE = "fr"  # Langue pour la recherche (fr, en, es, etc.)
LIMIT = 50       # Nombre de résultats par page (max 50)
DELAY = 2        # Délai entre les requêtes (secondes)

print(f"🌍 Langue de recherche: {LANGUAGE}")
print(f"📄 Résultats par page: {LIMIT}")
print(f"⏱️  Délai entre requêtes: {DELAY}s")

🌍 Langue de recherche: fr
📄 Résultats par page: 50
⏱️  Délai entre requêtes: 2s


## 🎯 Terme de recherche

Saisissez le terme que vous souhaitez rechercher dans Wikidata :

In [16]:
# Saisie du terme de recherche
search_term = input("🔍 Entrez le terme de recherche (par exemple, 'aero'): ")

if not search_term.strip():
    raise ValueError("❌ Le terme de recherche ne peut pas être vide.")


print(f"✅ Terme de recherche: '{search_term}'")

✅ Terme de recherche: 'aeronaut'


## 🛠️ Classe WikidataAeroScraper

Définition de la classe principale pour le scraping :

In [17]:
class WikidataAeroScraper:
    """Scraper pour récupérer toutes les entités Wikidata contenant un terme spécifique"""
    
    def __init__(self):
        self.endpoint = "https://query.wikidata.org/sparql"
        self.headers = {
            'User-Agent': 'WikidataBot/1.0 (https://github.com/May8326/wikidata-scraper)',
            'Accept': 'application/sparql-results+json'
        }
        self.results = []
        self.stats = {
            'total_pages': 0,
            'total_results': 0,
            'start_time': None,
            'end_time': None
        }

    def build_query(self, search_term, language, limit, offset):
        """Construit la requête SPARQL avec pagination"""
        query = f"""
        SELECT ?item ?itemLabel WHERE {{
          SERVICE wikibase:mwapi {{
            bd:serviceParam wikibase:endpoint "www.wikidata.org";
                            wikibase:api "EntitySearch";
                            mwapi:search "{search_term}";
                            mwapi:language "{language}";
                            mwapi:limit "{limit}";
                            mwapi:continue "{offset}".
            ?item wikibase:apiOutputItem mwapi:item.
          }}
          
          SERVICE wikibase:label {{ 
            bd:serviceParam wikibase:language "en". 
          }}
        }}
        """
        return query
    
    def execute_query(self, query):
        """Exécute une requête SPARQL et retourne les résultats"""
        try:
            response = requests.get(
                self.endpoint,
                params={'query': query, 'format': 'json'},
                headers=self.headers,
                timeout=30
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur lors de la requête: {e}")
            return None
    
    def extract_results(self, data):
        """Extrait les résultats du JSON retourné"""
        if not data or 'results' not in data:
            return []
        
        results = []
        for binding in data['results']['bindings']:
            item_uri = binding.get('item', {}).get('value', '')
            item_label = binding.get('itemLabel', {}).get('value', '')
            
            # Extraire l'ID Wikidata de l'URI
            item_id = item_uri.split('/')[-1] if item_uri else ''
            
            results.append({
                'wikidata_id': item_id,
                'label': item_label,
                'uri': item_uri
            })
        
        return results

print("✅ Classe WikidataAeroScraper définie avec succès!")

✅ Classe WikidataAeroScraper définie avec succès!


### Test pour un scraper plus robuste

## 🔄 Méthodes de scraping et sauvegarde

In [18]:
# Ajout des méthodes de scraping à la classe
def scrape_all_results(self):
    """Scrape tous les résultats en paginant automatiquement"""
    offset = 0
    page = 1
    total_results = 0
    
    self.stats['start_time'] = time.time()
    
    print(f"🚀 Début du scraping des entités contenant '{search_term}'...")
    print("=" * 60)
    
    while True:
        print(f"📄 Page {page} (offset: {offset})...", end=" ")
        
        # Construire et exécuter la requête
        query = self.build_query(search_term, LANGUAGE, LIMIT, offset)
        data = self.execute_query(query)
        
        if not data:
            print("❌ Erreur lors de l'exécution de la requête")
            break
        
        # Extraire les résultats
        page_results = self.extract_results(data)
        
        if not page_results:
            print(f"✅ Fin des résultats atteinte")
            break
        
        # Ajouter aux résultats totaux
        self.results.extend(page_results)
        total_results += len(page_results)
        
        print(f"[{len(page_results)} résultats] Total: {total_results}")
        
        # Vérifier si on a moins de LIMIT résultats (dernière page)
        if len(page_results) < LIMIT:
            print(f"✅ Dernière page atteinte ({len(page_results)} < {LIMIT} résultats)")
            break
        
        # Préparer la page suivante
        offset += LIMIT
        page += 1
        
        # Pause pour être respectueux envers l'API
        time.sleep(DELAY)
    
    self.stats['end_time'] = time.time()
    self.stats['total_pages'] = page
    self.stats['total_results'] = total_results
    
    duration = self.stats['end_time'] - self.stats['start_time']
    print("=" * 60)
    print(f"🎉 Scraping terminé! Total: {total_results} résultats en {duration:.1f}s")
    return self.results

def save_to_csv(self, filename=None):
    """Sauvegarde les résultats en CSV"""
    if not self.results:
        print("❌ Aucun résultat à sauvegarder")
        return
    
    if filename is None:
        filename = f"wikidata_{search_term}_results.csv"
    
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['wikidata_id', 'label', 'uri']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in self.results:
            writer.writerow(result)
    
    print(f"💾 Résultats sauvegardés dans: {filename}")
    return filename

def save_to_json(self, filename=None):
    """Sauvegarde les résultats en JSON"""
    if not self.results:
        print("❌ Aucun résultat à sauvegarder")
        return
    
    if filename is None:
        filename = f"wikidata_{search_term}_results.json"
    
    with open(filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(self.results, jsonfile, ensure_ascii=False, indent=2)
    
    print(f"💾 Résultats sauvegardés dans: {filename}")
    return filename

def get_dataframe(self):
    """Retourne les résultats sous forme de DataFrame pandas"""
    if not self.results:
        return pd.DataFrame()
    
    return pd.DataFrame(self.results)

def print_sample_results(self, limit=10):
    """Affiche un échantillon des résultats"""
    if not self.results:
        print("❌ Aucun résultat à afficher")
        return
    
    print(f"\n📋 Échantillon des résultats (premiers {limit}):")
    print("-" * 80)
    
    for i, result in enumerate(self.results[:limit]):
        print(f"{i+1:2d}. {result['wikidata_id']:12s} | {result['label']}")
    
    if len(self.results) > limit:
        print(f"... et {len(self.results) - limit} autres résultats")

def get_stats(self):
    """Retourne les statistiques du scraping"""
    return self.stats

# Ajouter les méthodes à la classe
WikidataAeroScraper.scrape_all_results = scrape_all_results
WikidataAeroScraper.save_to_csv = save_to_csv
WikidataAeroScraper.save_to_json = save_to_json
WikidataAeroScraper.get_dataframe = get_dataframe
WikidataAeroScraper.print_sample_results = print_sample_results
WikidataAeroScraper.get_stats = get_stats

print("✅ Méthodes de scraping et sauvegarde ajoutées!")

✅ Méthodes de scraping et sauvegarde ajoutées!


## 🚀 Exécution du scraping

Lance le processus de scraping complet :

In [19]:
# Initialisation du scraper
scraper = WikidataAeroScraper()

try:
    # Lancer le scraping
    results = scraper.scrape_all_results()
    
    if results:
        print(f"\n🎯 Mission accomplie! {len(results)} entités récupérées.")
    else:
        print("❌ Aucun résultat trouvé")
        
except KeyboardInterrupt:
    print("\n⛔ Arrêt demandé par l'utilisateur")
    if scraper.results:
        print(f"💾 Sauvegarde des {len(scraper.results)} résultats partiels...")
        scraper.save_to_csv(f"wikidata_{search_term}_partial.csv")
except Exception as e:
    print(f"❌ Erreur inattendue: {e}")

🚀 Début du scraping des entités contenant 'aeronaut'...
📄 Page 1 (offset: 0)... ❌ Erreur lors de la requête: 500 Server Error: Internal Server Error for url: https://query.wikidata.org/sparql?query=%0A++++++++SELECT+%3Fitem+%3FitemLabel+WHERE+%7B%0A++++++++++SERVICE+wikibase%3Amwapi+%7B%0A++++++++++++bd%3AserviceParam+wikibase%3Aendpoint+%22www.wikidata.org%22%3B%0A++++++++++++++++++++++++++++wikibase%3Aapi+%22EntitySearch%22%3B%0A++++++++++++++++++++++++++++mwapi%3Asearch+%22aeronaut%22%3B%0A++++++++++++++++++++++++++++mwapi%3Alanguage+%22fr%22%3B%0A++++++++++++++++++++++++++++mwapi%3Alimit+%2250%22%3B%0A++++++++++++++++++++++++++++mwapi%3Acontinue+%220%22.%0A++++++++++++%3Fitem+wikibase%3AapiOutputItem+mwapi%3Aitem.%0A++++++++++%7D%0A%0A++++++++++SERVICE+wikibase%3Alabel+%7B+%0A++++++++++++bd%3AserviceParam+wikibase%3Alanguage+%22en%22.+%0A++++++++++%7D%0A++++++++%7D%0A++++++++&format=json
❌ Erreur lors de l'exécution de la requête
🎉 Scraping terminé! Total: 0 résultats en 10.3s
❌ Au

### Débogage du scraping

In [ ]:
# Cellule de débogage pour identifier la cause de l'arrêt
def debug_scraping_stop(scraper):
    """Diagnostique pourquoi le scraping s'est arrêté"""
    print("🔍 DIAGNOSTIC DE L'ARRÊT DU SCRAPING")
    print("=" * 50)
    
    # Vérifier les statistiques
    stats = scraper.get_stats()
    total_results = len(scraper.results)
    
    print(f"📊 Résultats obtenus: {total_results}")
    print(f"📄 Pages scrapées: {stats.get('total_pages', 'N/A')}")
    
    # Vérifier la dernière page
    if total_results > 0:
        last_offset = (stats.get('total_pages', 1) - 1) * LIMIT
        print(f"🔢 Dernier offset utilisé: {last_offset}")
        
        # Test de la page suivante pour voir s'il y a encore des résultats
        print(f"\n🧪 Test de la page suivante (offset {last_offset + LIMIT})...")
        
        next_query = scraper.build_query(search_term, LANGUAGE, LIMIT, last_offset + LIMIT)
        next_data = scraper.execute_query(next_query)
        
        if next_data:
            next_results = scraper.extract_results(next_data)
            print(f"   → {len(next_results)} résultats trouvés sur la page suivante")
            
            if len(next_results) == 0:
                print("   ✅ Confirmation: plus de résultats disponibles")
            else:
                print("   ⚠️  Il y a encore des résultats ! Le scraping s'est arrêté prématurément")
                
                # Afficher quelques exemples de la page suivante
                print("   📋 Exemples de résultats manqués:")
                for i, result in enumerate(next_results[:5]):
                    print(f"      {i+1}. {result['wikidata_id']} | {result['label']}")
        else:
            print("   ❌ Erreur lors du test de la page suivante")
    
    # Vérifier s'il y a une limite exacte à 10000
    if total_results == 10000:
        print(f"\n⚠️  ALERTE: Arrêt à exactement 10 000 résultats!")
        print(f"   Cela suggère une limite API côté Wikidata EntitySearch")
    
    print("=" * 50)

# Lancer le diagnostic
if 'scraper' in locals() and scraper.results:
    debug_scraping_stop(scraper)
else:
    print("❌ Aucun scraper avec des résultats trouvé")



🔍 DIAGNOSTIC DE L'ARRÊT DU SCRAPING
📊 Résultats obtenus: 22
📄 Pages scrapées: 1
🔢 Dernier offset utilisé: 0

🧪 Test de la page suivante (offset 50)...
❌ Erreur lors de la requête: 500 Server Error: Internal Server Error for url: https://query.wikidata.org/sparql?query=%0A++++++++SELECT+%3Fitem+%3FitemLabel+WHERE+%7B%0A++++++++++SERVICE+wikibase%3Amwapi+%7B%0A++++++++++++bd%3AserviceParam+wikibase%3Aendpoint+%22www.wikidata.org%22%3B%0A++++++++++++++++++++++++++++wikibase%3Aapi+%22EntitySearch%22%3B%0A++++++++++++++++++++++++++++mwapi%3Asearch+%22aero%22%3B%0A++++++++++++++++++++++++++++mwapi%3Alanguage+%22fr%22%3B%0A++++++++++++++++++++++++++++mwapi%3Alimit+%2250%22%3B%0A++++++++++++++++++++++++++++mwapi%3Acontinue+%2250%22.%0A++++++++++++%3Fitem+wikibase%3AapiOutputItem+mwapi%3Aitem.%0A++++++++++%7D%0A%0A++++++++++SERVICE+wikibase%3Alabel+%7B+%0A++++++++++++bd%3AserviceParam+wikibase%3Alanguage+%22en%22.+%0A++++++++++%7D%0A++++++++%7D%0A++++++++&format=json
   ❌ Erreur lors du test de

### Essai pour contourner les limites

In [ ]:
def alternative_search_all_results(search_terme, max_retries=3):
    """
    Méthode alternative utilisant plusieurs approches combinées
    pour récupérer TOUS les résultats
    """
    print(f"🔄 Recherche alternative pour '{search_terme}'...")
    
    # Méthode 1: EntitySearch classique (jusqu'à 10k)
    scraper1 = WikidataAeroScraper()
    results1 = scraper1.scrape_all_results()
    
    print(f"📊 Méthode 1 (EntitySearch): {len(results1)} résultats")
    
    # Méthode 2: Recherche SPARQL directe avec REGEX
    query_sparql = f"""
    SELECT DISTINCT ?item ?itemLabel WHERE {{
      ?item rdfs:label ?itemLabel .
      FILTER(LANG(?itemLabel) = "en")
      FILTER(REGEX(?itemLabel, "{search_terme}", "i"))
      
      SERVICE wikibase:label {{ 
        bd:serviceParam wikibase:language "en". 
      }}
    }}
    LIMIT 50000
    """
    
    print(f"🔄 Méthode 2: Recherche SPARQL directe...")
    
    headers = {
        'User-Agent': 'WikidataBot/1.0 (https://github.com/May8326/wikidata-scraper)',
        'Accept': 'application/sparql-results+json'
    }
    
    try:
        response = requests.get(
            "https://query.wikidata.org/sparql",
            params={'query': query_sparql, 'format': 'json'},
            headers=headers,
            timeout=60
        )
        
        if response.status_code == 200:
            data = response.json()
            results2 = []
            
            for binding in data.get('results', {}).get('bindings', []):
                item_uri = binding.get('item', {}).get('value', '')
                item_label = binding.get('itemLabel', {}).get('value', '')
                item_id = item_uri.split('/')[-1] if item_uri else ''
                
                results2.append({
                    'wikidata_id': item_id,
                    'label': item_label,
                    'uri': item_uri,
                    'method': 'sparql_direct'
                })
            
            print(f"📊 Méthode 2 (SPARQL): {len(results2)} résultats")
        else:
            print(f"❌ Méthode 2 échouée: {response.status_code}")
            results2 = []
            
    except Exception as e:
        print(f"❌ Erreur Méthode 2: {e}")
        results2 = []
    
    # Fusionner et dédupliquer les résultats
    all_results = results1.copy()
    
    # Ajouter les résultats de la méthode 2 qui ne sont pas déjà présents
    existing_ids = {r['wikidata_id'] for r in results1}
    
    new_from_method2 = 0
    for result in results2:
        if result['wikidata_id'] not in existing_ids:
            all_results.append(result)
            existing_ids.add(result['wikidata_id'])
            new_from_method2 += 1
    
    print(f"✅ Total combiné: {len(all_results)} résultats")
    print(f"   → {len(results1)} de EntitySearch")
    print(f"   → {new_from_method2} nouveaux de SPARQL direct")
    
    return all_results

# Utilisation
# all_results = alternative_search_all_results(search_terme)

## 📱 Affichage avec DataFrame pandas

Visualisation moderne avec pandas :

In [12]:
# Conversion en DataFrame pandas
if scraper.results:
    df = scraper.get_dataframe()
    
    print(f"📊 DataFrame avec {len(df)} entrées:")
    print(f"Colonnes: {list(df.columns)}")
    
    # Affichage des premières lignes
    display(df.head(10))
    
    # Informations sur le dataset
    print(f"\n📋 Informations sur le dataset:")
    print(df.info())
else:
    print("❌ Aucune donnée à afficher")

📊 DataFrame avec 22 entrées:
Colonnes: ['wikidata_id', 'label', 'uri']


,wikidata_id,label,uri
0,Q35684,TARDIS,http://www.wikidata.org/entity/Q35684
1,Q565483,3325 TARDIS,http://www.wikidata.org/entity/Q565483
2,Q30897888,traversable acausal retrograde domains in spac...,http://www.wikidata.org/entity/Q30897888
3,Q29043590,Doctor Who: TARDIS,http://www.wikidata.org/entity/Q29043590
4,Q7685721,Tardisode,http://www.wikidata.org/entity/Q7685721
5,Q659822,Mastrils,http://www.wikidata.org/entity/Q659822
6,Q50403238,Traversable acausal retrograde domains in spac...,http://www.wikidata.org/entity/Q50403238
7,Q124372683,TARDIS,http://www.wikidata.org/entity/Q124372683
8,Q106601446,Tardis Data Core,http://www.wikidata.org/entity/Q106601446
9,Q57655398,TARDIS,http://www.wikidata.org/entity/Q57655398



📋 Informations sur le dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   wikidata_id  22 non-null     object
 1   label        22 non-null     object
 2   uri          22 non-null     object
dtypes: object(3)
memory usage: 660.0+ bytes
None


## 💾 Sauvegarde des résultats

Export des données dans différents formats :

In [13]:
# Sauvegarde des résultats
if scraper.results:
    # Sauvegarde CSV
    csv_file = scraper.save_to_csv()
    
    # Sauvegarde JSON
    json_file = scraper.save_to_json()
    
    # Sauvegarde Excel avec pandas (optionnel)
    try:
        df = scraper.get_dataframe()
        excel_file = f"wikidata_{search_term}_results.xlsx"
        df.to_excel(excel_file, index=False)
        print(f"💾 Résultats sauvegardés dans: {excel_file}")
    except ImportError:
        print("⚠️  Excel non disponible (installer openpyxl: pip install openpyxl)")
    
    print(f"\n✅ Tous les fichiers ont été sauvegardés!")
else:
    print("❌ Aucun résultat à sauvegarder")

💾 Résultats sauvegardés dans: wikidata_tardis_results.csv
💾 Résultats sauvegardés dans: wikidata_tardis_results.json
⚠️  Excel non disponible (installer openpyxl: pip install openpyxl)

✅ Tous les fichiers ont été sauvegardés!


## 🔍 Analyse des résultats

Quelques analyses simples des données récupérées :

In [14]:
# Analyses des résultats
if scraper.results:
    df = scraper.get_dataframe()
    
    print("🔍 Analyse des résultats:")
    print(f"   • Nombre total d'entités: {len(df)}")
    print(f"   • Nombre d'IDs uniques: {df['wikidata_id'].nunique()}")
    print(f"   • Nombre de labels uniques: {df['label'].nunique()}")
    
    # Labels les plus longs/courts
    df['label_length'] = df['label'].str.len()
    print(f"   • Label le plus long: {df['label_length'].max()} caractères")
    print(f"   • Label le plus court: {df['label_length'].min()} caractères")
    
    # Top 10 des labels les plus longs
    print(f"\n📏 Top 5 des labels les plus longs:")
    longest = df.nlargest(5, 'label_length')[['wikidata_id', 'label', 'label_length']]
    for _, row in longest.iterrows():
        print(f"   • {row['wikidata_id']}: {row['label']} ({row['label_length']} chars)")
    
    # Recherche de mots-clés spécifiques dans les labels
    keywords = ['company', 'airport', 'aircraft', 'airline']
    print(f"\n🏷️  Analyse par mots-clés:")
    for keyword in keywords:
        count = df['label'].str.contains(keyword, case=False, na=False).sum()
        if count > 0:
            print(f"   • '{keyword}': {count} entités")
else:
    print("❌ Aucune donnée à analyser")

🔍 Analyse des résultats:
   • Nombre total d'entités: 22
   • Nombre d'IDs uniques: 22
   • Nombre de labels uniques: 19
   • Label le plus long: 51 caractères
   • Label le plus court: 6 caractères

📏 Top 5 des labels les plus longs:
   • Q30897888: traversable acausal retrograde domains in spacetime (51 chars)
   • Q50403238: Traversable acausal retrograde domains in spacetime (51 chars)
   • Q99441542: TarDiS: Targets and Dynamics in Speech (38 chars)
   • Q29043590: Doctor Who: TARDIS (18 chars)
   • Q119844953: Tardis in a Field (17 chars)

🏷️  Analyse par mots-clés:


## 🎯 Résumé final

Bilan de l'exécution :

In [16]:
# Résumé final
print("🎯 RÉSUMÉ FINAL")
print("=" * 50)
print(f"Terme recherché: '{search_term}'")
print(f"Langue: {LANGUAGE}")
print(f"Résultats trouvés: {len(scraper.results) if scraper.results else 0}")

if scraper.results:
    stats = scraper.get_stats()
    if stats['start_time'] and stats['end_time']:
        duration = stats['end_time'] - stats['start_time']
        print(f"Durée d'exécution: {duration:.1f} secondes")
        print(f"Pages scrapées: {stats['total_pages']}")
    
    print(f"\n📁 Fichiers générés:")
    print(f"   • wikidata_{search_term}_results.csv")
    print(f"   • wikidata_{search_term}_results.json")
    print(f"   • wikidata_{search_term}_results.xlsx (si disponible)")
    
    print(f"\n✅ Scraping terminé avec succès!")
else:
    print(f"\n❌ Aucun résultat trouvé pour '{search_term}'")

print("=" * 50)

🎯 RÉSUMÉ FINAL
Terme recherché: 'tardis'
Langue: fr
Résultats trouvés: 22
Durée d'exécution: 0.4 secondes
Pages scrapées: 1

📁 Fichiers générés:
   • wikidata_tardis_results.csv
   • wikidata_tardis_results.json
   • wikidata_tardis_results.xlsx (si disponible)

✅ Scraping terminé avec succès!
